### Estatísticas em Python

#### Representação e interação de dados

##### Dados como uma tabela

O cenário que consideramos para análise estatística é o de múltiplas observações ou amostras descritas por um conjunto de diferentes atributos ou características. Os dados podem então ser vistos como uma tabela 2D, ou matriz, com colunas dando os diferentes atributos dos dados e linhas as observações. Por exemplo, os dados contido em data/brain_size.csv:

##### O quadro de dados do pandas

Criando dataframes: lendo arquivos de dados ou convertendo arrays

Leitura de um arquivo CSV: usando o arquivo CSV acima que fornece observações do tamanho e peso do cérebro e QI (Willerman et al. 1991), os dados são uma mistura de valores numéricos e categóricos:

In [ ]:
import pandas

dados = pandas.read_csv('data/brain_size.csv', 
                 sep=';',
                 na_values=".")

dados

Criando a partir de arrays: Um pandas.DataFrame também pode ser visto como um dicionário de 'séries' 1D, por exemplo, arrays ou listas. Se tivermos 3 arrays numpy:

In [ ]:
import numpy as np

t = np.linspace(-6, 6, 20)
sin_t = np.sin(t)
cos_t = np.cos(t)

Podemos expô-los como pandas.DataFrame:

In [ ]:
pandas.DataFrame({'t': t, 
                  'sin': sin_t,
                  'cos': cos_t})

Manipulação de dados

data é um pandas.DataFrame, que se assemelha ao dataframe de R:

In [ ]:
dados.shape # 40 linhas e 8 colunas

In [ ]:
dados.columns # Tem colunas

In [ ]:
print(dados['Gender']) # As colunas podem ser endereçadas pelo nome

In [ ]:
# Seletor mais simples
dados[dados['Gender'] == 'Female']['VIQ'].mean()

groupby: dividindo um dataframe em valores de variáveis categóricas:

In [ ]:
groupby_gender = dados.groupby('Gender')
for gender, value in groupby_gender['VIQ']:
    print((gender, value.mean()))

groupby_gender é um objeto poderoso que expõe muitas operações no grupo resultante de dataframes:

In [ ]:
groupby_gender.mean()

##### Plotando dados

Pandas vem com algumas ferramentas de plotagem (pandas.tools.plotting, usando matplotlib nos bastidores) para exibir estatísticas dos dados em dataframes:

Matrizes de dispersão:

In [ ]:
from pandas import plotting

plotting.scatter_matrix(dados[['Weight', 'Height', 'MRI_Count']]);

In [ ]:
plotting.scatter_matrix(dados[['PIQ', 'VIQ', 'FSIQ']]);

Duas populações

As métricas de QI são bimodais, como se existissem 2 subpopulações.

#### Teste de hipótese: comparando dois grupos

Para testes estatísticos simples, usaremos o submódulo scipy.stats do scipy:

In [ ]:
from scipy import stats

#### Teste t de Student: o teste estatístico mais simples

Teste t de 1 amostra: testa o valor de uma média populacional

scipy.stats.ttest_1samp() testa se a média populacional dos dados provavelmente será igual a um determinado valor (tecnicamente, se as observações forem extraídas de distribuições gaussianas de uma dada média populacional). Ele retorna a estatística T e o valor p (consulte a ajuda da função):

In [ ]:
stats.ttest_1samp(dados['VIQ'], 0)

Teste t de 2 amostras: testar diferenças entre populações

Vimos acima que os VIQs médios nas populações masculina e feminina eram diferentes. Para testar se isso é significativo, fazemos um teste t de 2 amostras com scipy.stats.ttest_ind():

In [ ]:
female_viq = dados[dados['Gender'] == 'Female']['VIQ']
male_viq = dados[dados['Gender'] == 'Male']['VIQ']
stats.ttest_ind(female_viq, male_viq)

##### Testes pareados: medidas repetidas nos mesmos indivíduos

PIQ, VIQ e FSIQ fornecem 3 medidas de QI. vamos testar se FISQ e PIQ forem significativamente diferentes. Podemos usar um teste de 2 amostras:

In [ ]:
stats.ttest_ind(dados['FSIQ'], dados['PIQ'])

O problema com esta abordagem é que ela esquece que existem ligações entre as observações: FSIQ e PIQ são medidos nos mesmos indivíduos. Assim, a variância devido à variabilidade intersujeitos é confusão, e pode ser removido, usando um “teste pareado” ou “teste de medidas repetidas”:

In [ ]:
stats.ttest_rel(dados['FSIQ'], dados['PIQ'])

Isso é equivalente a um teste de 1 amostra na diferença:

In [ ]:
stats.ttest_1samp(dados['FSIQ'] - dados['PIQ'], 0)

Os testes t assumem erros gaussianos. Podemos usar um teste de posto sinalizado de Wilcoxon, que relaxa essa suposição:

In [ ]:
stats.wilcoxon(dados['FSIQ'], dados['PIQ'])

#### Modelos lineares, fatores múltiplos e análise de variância

##### “fórmulas” para especificar modelos estatísticos em Python

Uma regressão linear simples

Primeiro, geramos dados simulados de acordo com o modelo:

In [ ]:
import numpy as np

x = np.linspace(-5, 5, 20)
np.random.seed(1)

# ruído distribuído normal
y = -5 + 3*x + 4 * np.random.normal(size=x.shape)

# Crie um quadro de dados contendo todas as variáveis relevantes
data = pandas.DataFrame({'x': x, 'y': y})

“fórmulas” para estatísticas em Python

Em seguida, especificamos um modelo OLS e o ajustamos:

In [ ]:
from statsmodels.formula.api import ols
model = ols("y ~ x", data).fit()

Podemos inspecionar as várias estatísticas derivadas do ajuste:

In [ ]:
print(model.summary())

Variáveis categóricas: comparando grupos ou várias categorias

Voltemos aos dados sobre o tamanho do cérebro:

In [ ]:
data = pandas.read_csv('data/brain_size.csv', 
                       sep=';',
                       na_values=".")

Podemos escrever uma comparação entre o QI de homens e mulheres usando um modelo linear:

In [ ]:
model = ols("VIQ ~ Gender + 1", data).fit()
print(model.summary())

In [ ]:
data_fisq = pandas.DataFrame({'iq': data['FSIQ'], 'type': 'fsiq'})
data_piq = pandas.DataFrame({'iq': data['PIQ'], 'type': 'piq'})
data_long = pandas.concat((data_fisq, data_piq))

print(data_long)

In [ ]:
model = ols("iq ~ type", data_long).fit()
print(model.summary())

In [ ]:
stats.ttest_ind(data['FSIQ'], data['PIQ'])

##### Regressão Múltipla: incluindo múltiplos fatores

In [ ]:
data = pandas.read_csv('examples/iris.csv')
model = ols('sepal_width ~ name + petal_length', data).fit()
print(model.summary())

##### Teste de hipótese post-hoc: análise de variância (ANOVA)

No exemplo da íris acima, queremos testar se o comprimento da pétala é diferente entre versicolor e virginica, depois de remover o efeito da largura da sépala. Isso pode ser formulado como um teste da diferença entre o coeficiente associado a versicolor e virginica no modelo linear estimado acima (é uma Análise de Variância, ANOVA). Para isso, escrevemos um vetor de ‘contraste’ nos parâmetros estimados: queremos para testar "nome[T.versicolor] - nome[T.virginica]", com um teste F:

In [ ]:
print(model.f_test([0, 1, -1, 0]))

#### Mais visualização: seaborn para exploração estatística

Seaborn combina ajustes estatísticos simples com plotagem em dataframes de pandas.
Consideremos dados de salários e muitas outras informações pessoais de 500 indivíduos (Berndt, ER. A Prática da Econometria. 1991. NY: Addison-Wesley).

In [ ]:
print(data)

##### Pairplot: matrizes de dispersão

Podemos facilmente ter uma intuição sobre as interações entre variáveis contínuas usando seaborn. pairplot() para exibir uma matriz de dispersão:

In [ ]:
import seaborn

seaborn.pairplot(data,
                 vars=['WAGE', 'AGE', 'EDUCATION'],
                 kind='reg')

Variáveis categóricas podem ser plotado como o matiz:

In [ ]:
seaborn.pairplot(data, 
                 vars=['WAGE', 'AGE', 'EDUCATION'],
                 kind='reg', hue='SEX')

---

Configurações de aparência e matplotlib

A Seaborn altera o padrão das figuras do matplotlib para obter uma aparência mais “moderna”, “semelhante ao Excel”. Isto faz isso na importação. Você pode redefinir o padrão usando:

In [ ]:
from matplotlib import pyplot as plt
plt.rcdefaults()

Dica: para voltar às configurações marítimas ou entender melhor o estilo de navegação marítima, consulte a seção relevante da documentação marítima.

---

##### lmplot: traçando uma regressão univariada

Uma regressão que captura a relação entre uma variável e outra, por exemplo, salário e educação, pode ser plotada usando seaborn.lmplot():

In [ ]:
seaborn.lmplot(y='WAGE', x='EDUCATION', data=data)

#### Teste de interações

Os salários aumentam mais com a educação para os homens do que para as mulheres?

In [ ]:
result = sm.ols(formula='wage ~ education + gender + education * gender',
                data=data).fit()
print(result.summary())

#### Código completo das figuras

##### Boxplots e diferenças pareadas

Traçar boxplots para FSIQ, PIQ e a diferença emparelhada entre os dois: enquanto o spread (barras de erro) para FSIQ e PIQ são muito grandes, há um efeito sistemático (comum) devido aos sujeitos. este efeito é anulado na diferença e o spread da diferença (“pareado” por sujeito) é bem menor do que a propagação das medidas individuais.

In [ ]:
import pandas
import matplotlib.pyplot as plt

data = pandas.read_csv('brain_size.csv', sep=';', na_values='.')

# Box plot de FSIQ e PIQ (diferentes medidas de QI)
plt.figure(figsize=(4, 3))
data.boxplot(column=['FSIQ', 'PIQ'])

# Boxplot da diferença
plt.figure(figsize=(4, 3))
plt.boxplot(data['FSIQ'] - data['PIQ'])
plt.xticks((1, ), ('FSIQ - PIQ', ))
plt.show()